In [ ]:
# DEPENDENCIES
!pip install mtcnn
!pip install fastapi uvicorn
!pip install colabcode
#!pip install sklearn
#!pip install tensorflow
#!pip install numpy
#!pip install pandas
#!pip install tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# IMPORTING LIBs
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import os
import random
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

from PIL import Image
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from keras.models import load_model

from mtcnn.mtcnn import MTCNN

from sklearn.base import BaseEstimator, TransformerMixin

from fastapi import FastAPI, File, UploadFile
import uvicorn
from colabcode import ColabCode

0it [00:00, ?it/s]

In [ ]:
## DATA PATHS 

#ROOT_PATH = '/content/drive/MyDrive/FaceNet_API/ID_DATA/'
TRAIN_PATH = '/content/drive/MyDrive/FaceNet_API/ID_DATA/train/'
VAL_PATH = '/content/drive/MyDrive/FaceNet_API/ID_DATA/test/'

In [ ]:
# Following processes happen
# IMG -> IDENTIFY FACES -> NP_ARRAY -> FACE_EMBEDDING -> NP_ARRAY

class load_prep_data(BaseEstimator, TransformerMixin):

  def __init__(self):
    print('---> init called')
    # Pre-trained model by Hiroki Taniai
    self.model = load_model('/content/drive/MyDrive/FaceNet_API/Models/facenet_keras.h5')
    print('---> Loaded model')
    
  # extract a single face from a given photograph
  # EXTRACT FACES
  def extract_face(self,filename, required_size=(160, 160)):
      image = Image.open(filename)
      image = image.convert('RGB')
      pixels = np.asarray(image)
      
      # detect faces using MTCNN
      detector = MTCNN()
      results = detector.detect_faces(pixels)
      
      # extract the bounding box from the first face
      try:
          x1, y1, width, height = results[0]['box']
      
          # bug fix
          x1, y1 = abs(x1), abs(y1)

          x2, y2 = x1 + width, y1 + height

          face = pixels[y1:y2, x1:x2]

          image = Image.fromarray(face)

          image = image.resize(required_size)
          face_array = np.asarray(image)

          return face_array
      except Exception as e:
          print(e)
          print(filename)

      # required_size= (160,160)
      image = image.resize(required_size)
      face_array = np.asarray(image)
      return face_array

  # Make a list of faces extracted from each image of each folder
  def load_faces(self,directory):
    faces = list()
    
    for file_name in tqdm(os.listdir(directory)):
        path = directory + file_name
        
        face = self.extract_face(path)
        
        faces.append(face)
    return faces

  # Get embedding of face array using pre-trained model by Hiroki Taniai
  def get_embedding(self,face_pixels):
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global) 
    mean, std = face_pixels.mean(), face_pixels.std() 
    face_pixels = (face_pixels - mean) / std
    
    samples = np.expand_dims(face_pixels, axis=0)
    
    yhat = self.model.predict(samples)
    #models = ["VGG-Face", "Facenet", "OpenFace", "DeepFace", "Dlib", "ArcFace"]
    #yhat = DeepFace.represent(face_pixels, model_name = models[1])
    
    return yhat[0]

  # Driver code to call above functions in one go
  # extract_face --> get_embedding --> reshaping the image
  def preprocess_img(self,file_path):
    inp = self.extract_face(file_path)  # EXTRACTS FACE FROM IMAGE
    inp_emb = self.get_embedding(inp) #MAKES EMBEDDING OF THE IMAGE USING PRE-TRAINED MODEL
    img = plt.imread(file_path)
    plt.axis('off')
    plt.imshow(img)
    return inp_emb.reshape(1,-1)

  def fit(self,X,y=None):
    print('---> fit called')
    return self

  ## Transforming data to face_embeddings and labels
  def transform(self,X,y=None):
    directory=X
    print('----> transform called')
    images, labels = list(), list()
    
    # Processes each image of each folder
    for folder in tqdm(os.listdir(directory)):
        
        print(folder)
        path = directory+'/'+folder+'/'
        if 'pdf' in path:
          continue

        if not os.path.isdir(path):
            continue

        # detect face from image
        faces = self.load_faces(path)
        if faces is None:
            continue

        print(f'EMPLOYEE : {folder}, Faces : {len(faces)}')

        # Append respective label of face to temporary list label:[List]
        label = [folder for _ in range(len(faces))]

        # Append faces:[List] and label:[List] array to images:[List], labels:[List]
        images.extend(faces)
        labels.extend(label)
        
    x_train,y_train= np.asarray(images), np.asarray(labels)

    # Loading pre-trained model
    print('loading model for extracting face embedding')

    # get face embedding for x_train
    trainX = list()

    # get embeddings of image
    for pixels in tqdm(x_train):
        if pixels is None:
            continue
        embedding = self.get_embedding(pixels)
        trainX.append(embedding)
    trainX = np.asarray(trainX)

    print('Train X :', trainX.shape)
    return trainX, y_train

In [ ]:
# NORMALIZE X DATA & LABEL ENCODE Y DATA

class norm_enc(BaseEstimator, TransformerMixin):
  def __init__(self):
    self.norm = Normalizer(norm='l2')
    self.le = LabelEncoder()
    print('->->->-> Initialised normaliser and label encoder')

  def fit(self,X,y=None):
    print('---> fit called')
    return self

  def transform(self,X,y=None):
    print('---> transform called')
    trainX = self.norm.transform(X[0])
    print('->->-> transformed x data')
    trainY = self.le.fit_transform(X[1])
    print('->-> encodede y data')
    return trainX,trainY,self.le

In [ ]:
# PIPELINE TO LOAD & TRANSFORM DATA

data_pipe = Pipeline(
    steps=[
           ('load',load_prep_data()),
           ('normalise & encode',norm_enc())
    ]
)

---> init called
---> Loaded model
->->->-> Initialised normaliser and label encoder


In [ ]:
# TRAIN, VAL DATA USING PIPELINE

x_train,y_train,label_train = data_pipe.fit_transform(TRAIN_PATH)
x_val,y_val,label_test = data_pipe.fit_transform(VAL_PATH)

---> fit called
----> transform called


  0%|          | 0/33 [00:00<?, ?it/s]

Alugoju Sathish


  0%|          | 0/2 [00:00<?, ?it/s]

EMPLOYEE : Alugoju Sathish, Faces : 2
Amol Dayanand Mane


...

EMPLOYEE : Vijay Arun Mane, Faces : 1
zareena


  0%|          | 0/1 [00:00<?, ?it/s]

EMPLOYEE : zareena, Faces : 1
loading model for extracting face embedding


  0%|          | 0/33 [00:00<?, ?it/s]

Train X : (33, 128)
---> fit called
---> transform called
->->-> transformed x data
->-> encodede y data


In [ ]:
# Pre trained
# Custom model
# Label Encoder file

In [ ]:

# PIPELINE TO TRAIN MODEL

model_pipe = Pipeline(
    steps=[
           ('SVC model',SVC(kernel='linear', probability=True))
    ]
)
model_pipe.fit(x_train,y_train)
#model_pipe.save('model_facenet_pipe.h5')

Pipeline(steps=[('SVC model', SVC(kernel='linear', probability=True))])

In [ ]:
# MODEL ACCURACY

accuracy = model_pipe.score(x_val,y_val)
print('model_accuracy:',accuracy)

model_accuracy: 0.7878787878787878


In [ ]:
# MODEL INFERENCE
le = LabelEncoder()

model_pipe.predict(x_val[0].reshape(1,-1))[0]

0

In [ ]:


selection = random.randint(0,x_val.shape[0])


yhat_prob = model_pipe.predict_proba(x_val[selection].reshape(1,-1))
yhat = model_pipe.predict(x_val[selection].reshape(1,-1))

pred = label_test.inverse_transform(yhat)
actual = label_test.inverse_transform(np.array([y_val[selection]]))

print(f'Probability : {np.max(yhat_prob)*100}, \nEmployee - Predicted : {pred}, Acutal : {actual}')

Probability : 7.782919808945509, 
Employee - Predicted : ['Vijay Arun Mane'], Acutal : ['Vijay Arun Mane']


In [ ]:
# Saving model
import pickle
pickle.dump(model_pipe, open('/content/drive/My Drive/model.pkl', 'wb'))

In [ ]:
# Saving labels
import numpy


numpy.save('/content/drive/My Drive/classes.npy', label_train.classes_)

In [ ]:
# Loading labels

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.classes_ = numpy.load('/content/drive/My Drive/classes.npy')